In [1]:
!pip install pandas
!pip install numpy 
!pip install vader-multi
!pip install gensim 
!pip install transvec 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 6.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def preprocFab():
    import pandas as pd
    import json
    import os

    import numpy as np
    import re #regex 
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

    import gensim
    from transvec.transformers import TranslationWordVectorizer
    import requests
    import os
    import zipfile

    #cheat code until the code can be totally checked by Fabrice
    df = pd.read_csv("clean_data_set_sentiment_only_for_english.csv", low_memory = False)
    return df

    #extract the data from the json files
    dfs = []
    for r, d, f in os.walk('input'):
        for file in f:
            if 'withheldtweets.json' in file:  # alt: if ‘control' in file:
                dfs.append(pd.read_json('%s/%s' % (r, file), lines=True))

    df_cen = pd.concat(dfs)
    df_cen = df_cen.dropna(subset=['withheld_in_countries'])


    #keep only the features that are worth keeping
    worthKeeping = ["text", "truncated", "user",
                "withheld_in_countries", "entities", "lang",
                "possibly_sensitive", "extended_tweet"]
    df_cen = df_cen[worthKeeping]


    #some tweets have NaN as "possibly sensitive"…
    df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)


    #recover the full text for truncated tweets
    dfRaw = df_cen.values
    for line in dfRaw:
        if not pd.isna(line[-1]):
            line[0] = line[-1]["full_text"]
            
        #remove urls from tweets
        #they are shortened anyway so we can't make use of them
        line[0] = re.sub(r'http\S+', '', line[0])
        
        #flatten retweets
        line[0] = re.sub(r'RT @\S+:', '', line[0])

    dfRaw = np.delete(dfRaw, len(worthKeeping)-1, axis=1) #remove "extended_tweet"
    worthKeeping.remove("extended_tweet")

    dfRaw = np.delete(dfRaw, 1, axis=1) #remove "truncated"
    worthKeeping.remove("truncated")


    #extract hashtags seperately
    for line in dfRaw:
        line[3] = [x["text"] for x in line[3]["hashtags"]]
    worthKeeping[3] = "hashtags"


    #create a feature for user-verified and user-followers_count
    verified = [line[1]["verified"] for line in dfRaw]
    followers = [line[1]["followers_count"] for line in dfRaw]


    #for the location, Rebekah suggested to only spot the country name and discard the rest
    listOfCountries = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']
    def findCountry(x): #the code for this function is self explanatory
        for country in listOfCountries:
            if x and country in x:
                return country
        return None

    location = [findCountry(line[1]["location"]) for line in dfRaw]

    dfRaw = np.c_[dfRaw, verified, followers, location]
    worthKeeping += ["verified_account", "followers_count", "location"]


    #binary feature for whether the tweet has been withheld anywhere
    withheld = []
    for line in dfRaw:
        if not isinstance(line[2], list):
            line[2] = []
        withheld.append(len(line[2]) != 0)
            
    dfRaw = np.c_[dfRaw, withheld]
    worthKeeping += ["withheld_anywhere"]


    #sentiment analysis
    #https://www.analyticsvidhya.com/blog/2022/07/sentiment-analysis-using-python/? with VADER
    #https://github.com/brunneis/vader-multi, same concept but multilingual
    #text gets translated into english and then sentiment analysis is applied to the english text
    #takes a LOT of time
    analyzer = SentimentIntensityAnalyzer()

    #we made the assumption that sentiment analysis for this analyzer only works for english
    res = np.array([[x for x in analyzer.polarity_scores(line[0]).values()] for line in dfRaw])

    dfRaw = np.c_[dfRaw, res]
    worthKeeping += ["neg", "neu", "pos", "compound"]


    #popularity feature:
    #build a score based on the values of followers_count, favourites_count, statuses_count
    #compute a score from 0 to 1 for each, with (x - min)/(max - min), then comptute the average of these scores 

    followers_count = np.array([line[1]["followers_count"] for line in dfRaw])
    favourites_count = np.array([line[1]["favourites_count"] for line in dfRaw])
    statuses_count = np.array([line[1]["statuses_count"] for line in dfRaw])

    def normalize(array):
        return (array - np.min(array)) / (np.max(array) - np.min(array))

    score = (1/3) * (normalize(followers_count) + normalize(favourites_count) + normalize(statuses_count))
    dfRaw = np.c_[dfRaw, score]
    worthKeeping += ["popularity_score"]


    #reassemble the data in a pandas dataframe and remove the column "user"
    df_cen = pd.DataFrame(dfRaw, columns = worthKeeping)
    cleanCols = filter(lambda x: x != "user", worthKeeping)
    df_clean = df_cen[cleanCols]


    #word2vec
    def download_model(url, filename, removeFile = True):
        """Download a word2vec model from the url"""
        #download the model
        if not os.path.exists(f"input/{filename}"):
            with open("input/model-temp.zip", "wb") as f: #download
                f.write(requests.get(url).content)
                
            with zipfile.ZipFile("input/model-temp.zip") as zf: #extract the data for the model
                zf.extract("model.bin", "./input/")

            os.rename("input/model.bin", f"input/{filename}")
            os.remove("input/model-temp.zip")

        #make it
        model = gensim.models.KeyedVectors.load_word2vec_format(f"input/{filename}", binary=True)
        
        if removeFile:
            os.remove(f"input/{filename}")
        
        return model

    def sentence2vec(sentence, model, split=True):
        """Compute the word2vec vector of the text of a tweet"""
        vector = None
        n = 0
        
        def removePunctNumbers(sentence):
            toRemove = "²&~\"#'{}'([])|-`\_^@+1234567890°$£µ*%!§:/;.,?<>€"
            for char in toRemove: sentence = sentence.replace(char, "")
            return sentence.lower()
        
        words = removePunctNumbers(sentence)
        if split: words = words.split(" ")
        else: words = list(words)

        for word in words:
            if word in model:
                if vector is None: vector = model[word]
                else: vector = vector + model[word]
                n += 1

        if n > 0:
            return vector / n
        else:
            return np.zeros(300, dtype=np.float32)

    #from http://vectors.nlpl.eu/repository/
    models = { #urls of the models in the repository
        "en": "http://vectors.nlpl.eu/repository/20/8.zip",
        "ru": "http://vectors.nlpl.eu/repository/20/184.zip",
        "fr": "http://vectors.nlpl.eu/repository/20/43.zip",
        "ur": "http://vectors.nlpl.eu/repository/20/72.zip",
        "es": "http://vectors.nlpl.eu/repository/20/68.zip",
        "ar": "http://vectors.nlpl.eu/repository/20/31.zip",
        "in": "http://vectors.nlpl.eu/repository/20/50.zip",
        "ja": "http://vectors.nlpl.eu/repository/20/53.zip",
        "ko": "http://vectors.nlpl.eu/repository/20/55.zip",
        "pt": "http://vectors.nlpl.eu/repository/20/63.zip",
        #there is no model for thai in this repository
        "tr": "http://vectors.nlpl.eu/repository/20/70.zip",
        #und is probably "undetermined language"
    }
    referrence_words = ["water", "sun", "sky", "king", "night", "year"]
    #referrence words are used to "rotate" the vector spaces of different
    #languages so that they become comparable to the one of the model for
    #english: we give those words and their translations so that the translation
    #has about the same vector as its english equivalent
    ref_trans = {
        "ru": ['вода', 'солнце', 'небо', 'король', 'сутки', 'год'],
        "fr": ["eau", "soleil", "ciel", "roi", "nuit", "année"],
        "ur": ['پانی', 'سورج', 'آسمان', 'بادشاہ', 'رات', 'سال'], #it looks reversed but it's not
        "es": ['agua', 'dom', 'cielo', 'rey', 'noche', 'año'],
        "ar": ["ماء", "الشمس", "سماء", "ملِك", "ليل", "عام"], #it looks reversed but it's not
        "in": ['panas', 'matahari', 'langit', 'raja', 'malam', 'tahun'],
        "ja": ['水', '日', '空', '王', '泊', '年'], 
        "ko": ['물', '일', '하늘', '왕', '박', '년'],
        "pt": ['água', 'eno', 'céu', 'cama', 'noite', 'ano'],
        "tr": ['su', 'güneş', 'gökyüzü', 'kral', 'gece', 'yıl']
    }


    #get the model for english
    en_model = download_model(models["en"], "model_en.bin", removeFile=False)

    #get the raw numpy array of the data and add 300 new features: the components of word2vec vectors
    data = df.values
    columns = list(df.columns)
    del df
    data = np.c_[data, np.zeros((data.shape[0], 300))]

    #apply sentence2vec to tweets in a certain lang
    def vectorizeLang(lang, model, data):
        for i, line in enumerate(data):
            text, langLine = line[0], line[3]
            if langLine == lang:
                data[i][-300:] = sentence2vec(text, model, lang != "ja")
                #japanese is different since vectors are defined for one symbol
                #rather than one set of characters separated with a space, like
                #for languages using the latin alphabet
                #so the tokenization is different
    
    #once it's done, it's time to do it for other languages
    #since the models aren't in the same vector space, we use TranslationWordVectorizer
    for lang in ref_trans:
        if lang not in models or "ru" == lang: continue
        print(lang)
        #download the model
        lang_model = download_model(models[lang], f"model_{lang}.bin", False)
        en_model = download_model(models["en"], "model_en.bin", False)
        print("ok")
        
        #time to compute the bilingual model -> model for lang that is in the same vector space
        #as the model of english
        train_ref = [(a, b) for a, b in zip(referrence_words, ref_trans[lang])]
        bilingual_model = TranslationWordVectorizer(en_model, lang_model).fit(train_ref)
        print("bilingual ok")
        
        #let's delete lang_model, it's no longer useful and takes a lot of memory
        del lang_model
        
        vectorizeLang(lang, bilingual_model, data)
        print("lang completed ---")
        
        del bilingual_model
    
    for lang in ['ru']: #russian is different, the type of words is included in the model
        print(lang)
        #download the model
        lang_model = download_model(models[lang], f"model_{lang}.bin", False)
        en_model = download_model(models["en"], "model_en.bin", False)
        lang_model.vocab = {k.split("_")[0]: v for k, v in lang_model.vocab.items()}
        print("ok")
        
        #time to compute the bilingual model -> model for lang that is in the same vector space
        #as the model of english
        train_ref = [(a, b) for a, b in zip(referrence_words, ref_trans[lang])]
        bilingual_model = TranslationWordVectorizer(en_model, lang_model).fit(train_ref)
        print("bilingual ok")
        
        #let's delete lang_model, it's no longer useful and takes a lot of memory
        del lang_model
        
        vectorizeLang(lang, bilingual_model, data)
        print("lang completed ---")

        del bilingual_model

    #make the word2vec vectors be a single column in the dataframe
    dataExceptVector, vector = data[:,:-300], data[:,-300:]
    vectors = list(vector[i] for i in range(vector.shape[0]))
    df = pd.DataFrame(dataExceptVector, columns=columns[:-300])
    df["vector"] = vectors

    df_clean.to_csv("clean_dataset.csv")
    return df_clean